<a href="https://colab.research.google.com/github/magnolia2001/Forest_Estimation/blob/main/notebooks/NNandCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

以下是针对掩膜数据形状为(height, width)

如果掩膜数据形状为(1, height, width)则移除掩膜图像中的通道维度使其形状变为(height, width)

In [ ]:
import numpy as np
import pandas as pd
import datetime, os, cv2
from matplotlib import pyplot as plt
from matplotlib.ticker import StrMethodFormatter
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error as mse, mean_absolute_error as mae, mean_absolute_percentage_error as mape
from keras.models import Sequential, load_model
from keras.layers import Dense, BatchNormalization, Dropout, InputLayer, Flatten, Conv2D, MaxPool2D, AveragePooling2D
from keras.callbacks import TensorBoard, ModelCheckpoint

# 制作标签数据和特征数据

# 窗口大小应为奇数，以保证标签在中间
size = 5 #define window size should be odd so that the label is in the middle
# 特征的形状，这里假设每个特征是一个大小为 (size, size) 的窗口，包含 90 个通道
shape = (90, size, size) #define shape of features
# np.ones(shape, dtype=None) 用于创建一个形状为 shape 的数组，并将所有元素初始化为 1.0
# 其中 shape：指定数组的形状，通常是一个整数或元组。 dtype：指定数组元素的数据类型（可选）。如果不指定，默认使用 float64 类型。
# labels1 用于存放标签数据（掩膜）, data1 用于存放提取的特征数据
# np.ones(1)返回的是一个只有一个元素的数组，其中该元素值为 1。
labels1 = np.ones(1) #array for labels
# 创建了一个数组，形状为 shape 即 (90, 5, 5) 的 NumPy 数组，并且所有的元素值都被初始化为 1.0 。
data1 = np.ones(shape) #array for features
# 扩展维度，便于后续拼接操作
data1 = np.expand_dims(data1, axis=0) #expand dimension to concatenate

# 遍历目录中的图像（假设有 20 张图像, 具体数量还需要根据自己的情况修改）
# 在 for j in range(20) 这个遍历过程中，区分 j < 10 和 j >= 10 的目的是为了处理不同的文件命名规则。
# 对于小于 10 的文件名，文件名是 "image_00X.npy"，其中 X 是单个数字（0 到 9）。
# 对于大于等于 10 的文件名，文件名是 "image_0XY.npy"，其中 XY 是两位数的数字（10 到 19）。
for j in range(20): #iterate over images in directory
  if j < 10:
    # 路径填写实际路径
    # 读取图像数据
    X = np.load('/content/images/image_00'+ str(j) + '.npy')
    # 读取掩膜数据
    y = np.load('/content/masks/mask_00'+ str(j) + '.npy')
    # 移除掩膜图像中的通道维度使其形状变为(height, width)
    y = y[0, :, :]  # 去掉通道维度，保留二维掩膜图像

    # 选择掩膜图像 y 中所有大于 0 的位置（即标签不为 0 的位置），并返回这些位置的索引。
    # indices 数组返回 N 个元素，其中 N 为掩膜图像 y 中所有大于 0 的元素个数。每个元素都是一个长度为 2 的行向量，表示符合条件元素的行列索引。
    # y > 0 是一个布尔条件，返回一个与 y 相同形状的布尔数组, 如果是大于 0，布尔值为 True，否则为 False。
    # np.argwhere() 是 NumPy 库中的一个函数，它返回数组中满足某个条件的所有索引（行列坐标），即满足条件的元素的坐标位置。
    # 这里 indices 是一个形状为 (N, 2) 的二维数组，每一行是 (y, x) 坐标. y 为行索引, x 为列索引
    indices = np.argwhere(y > 0) #select all values with label

    # indices_2d 是 indices 数组的一个切片，是一个 二维数组, 它包含了所有掩膜图像中标签值大于 0 的位置的 列索引。
    # 切片操作 indices[:, 1:] 就是提取所有行中的第二列（即 行 和 列 坐标中的 列索引）。
    # indices_2d = indices[:, 1:] #extract indices

    # 初始化一个数组 ind_y 来收集符合条件的标签位置。
    # np.ones(2) 会创建一个包含 2 个元素的数组，所有元素的值为 1. 。
    # .reshape(-1, 2) 将该数组的形状重塑为 (-1, 2)，表示按列数为 2 进行重塑，-1 表示自动计算行数。由于只有 2 个元素，这会将数组变成形状为 (1, 2) 的二维数组。
    ind_y = np.ones(2).reshape(-1,2) #array to collect indices

    # 遍历掩膜中的每个标签位置
    # for i in indices_2d: #iterate over indices
    for i in indices:
      # 提取图像块，并检查其形状。
      # size//2 表示 size 除以 2 的整数部分，用于确定图像块中心点到边界的距离。右端点的值之所以加 1 是因为区间是左闭右开的,所以加 1 保证能取到右端点.
      # 整个i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2)表达式计算出一个范围，用于选取以 (i[0], i[1])（标签的 y, x 坐标） 为中心，上下各延伸 size//2 个像素的区域。
      # i[0] 是当前标签位置的 y 坐标（行索引）。i[1] 是当前标签位置的 x 坐标（列索引）。
      # 利用 shape 确保当前窗口大小与指定的窗口大小一致
      if shape == X[:, i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2) + 1].shape: #select only features with the same shape because of labels at the image border
        temp = X[:, i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2) + 1] #save them temporary
        # 将 temp 的维度扩展一个维度，使得它变成一个形状为 (1, channels, size, size) 的四维数组。扩展维度的目的是为了能够将 temp 与其他提取的窗口进行拼接。
        temp2 = np.expand_dims(temp, axis=0) #expand dimension to concatenate
        # 拼接特征数据
        # data1 最终会变成 (num_samples, 90, 5, 5)，其中 num_samples 是提取的窗口数量。
        data1 = np.concatenate((data1, temp2), axis=0) #concatenation
        # 拼接标签索引
        # i.reshape(-1, 2) 会把 i 重新调整为一个形状为 (1, 2) 的二维数组
        # axis=0 表示在 第 0 维（行方向） 进行拼接，即新添加的行会被添加到原数组的最后。
        # ind_y 则是一个 (num_samples, 2) 的数组，每个样本对应一个标签位置的 (y, x) 坐标。
        ind_y = np.concatenate((ind_y, i.reshape(-1,2)), axis=0) #concatenation of index so that they have the same order and length as the features

    # 去掉第一个虚拟值
    # 初始时，ind_y 中的第一个元素是 np.ones(2).reshape(-1,2) 创建的虚拟数据。此步骤是将它移除，只保留实际的标签坐标。
    ind_y = ind_y[1:] #remove first dummy values
    # 提取所有的行索引
    indices_1 = ind_y[:, 0].astype(int)
    # 提取所有的列索引
    indices_2 = ind_y[:, 1].astype(int)
    # 提取标签值
    # 从这句代码应该可以看出原作者的掩膜图像的形状包含了通道维度,即形状为(1, height, width)
    # data_y = y[0, indices_1, indices_2] #extract labels
    data_y = y[indices_1, indices_2]  # 提取标签
    # 拼接标签，形成最终的标签数组。
    labels1 = np.concatenate((labels1, data_y), axis = 0) #concatenate labels

  if j >= 10:
    X = np.load('/content/images/image_0'+ str(j) + '.npy')
    y = np.load('/content/masks/mask_0'+ str(j) + '.npy')
    # 移除掩膜图像中的通道维度使其形状变为(height, width)
    y = y[0, :, :]  # 去掉通道维度，保留二维掩膜图像
    indices = np.argwhere(y > 0)
    # indices_2d = indices[:, 1:]
    ind_y = np.ones(2).reshape(-1,2)
    # for i in indices_2d:
    for i in indices:
      if shape == X[:, i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2) + 1].shape:
        temp = X[:, i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2) + 1]
        temp2 = np.expand_dims(temp, axis=0)
        data1 = np.concatenate((data1, temp2), axis=0)

        ind_y = np.concatenate((ind_y, i.reshape(-1,2)), axis=0)

    # 去掉第一个虚拟值
    ind_y = ind_y[1:]
    # 提取所有的行索引
    indices_1 = ind_y[:, 0].astype(int)
    # 提取所有的列索引
    indices_2 = ind_y[:, 1].astype(int)
    # data_y = y[0, indices_1, indices_2]
    # 提取标签值
    # 每一对 (indices_1[i], indices_2[i]) 会自动匹配，得到对应位置的标签值。
    data_y = y[indices_1, indices_2]  # 提取标签
    # 拼接标签，形成最终的标签数组。
    labels1 = np.concatenate((labels1, data_y), axis = 0)

# 移除第一个虚拟值
# data1 的形状会是 (num_samples, 90, 5, 5)，其中 num_samples 是提取的窗口数量（即符合条件的标签数量）。一个四维数组
# labels1 的形状会是 (num_samples,)，其中 num_samples 是所有图像中符合条件的标签数量。一个一维数组
data1 = data1[1:] #remove first dummy values
labels1 = labels1[1:] #remove first dummy values

# data1 和 labels1 应该是 一一对应的，因为它们的样本数（num_samples）相同。
# 获得标签数据和特征数据
features = data1
labels = labels1


### Optional: Create equally distributed data set

对数据进行采样，平衡各个标签类的样本数量。它通过将不同标签范围的样本分成多个区间（每3米为一个区间），并对每个区间进行随机抽样，最终创建一个平衡的数据集。

In [ ]:
# 每个类别的样本数，确保每个标签区间有 800 个样本
sample_size = 800 #every class with labels smaller 36 meters has over 800 values
#features = np.mean(features, axis=(2, 3)) # patch mean of size * size features

# 生成从 3 到 36 步长为 3 的数字列表，即 [3, 6, 9, ..., 36]
num = (list(range(3, 37, 3))) #create list from 3 to 36 step 3
# 假设每个样本是一个 5x5 的图像块（大小为 5x5，90 个通道）
shape = (90, 5, 5)
# 创建一个初始的数组用于存储特征数据，形状为 (90, 5, 5)
data_bal = np.ones(shape) #create array to fill with features
# 扩展维度，使得形状变为 (1, 90, 5, 5)，这样可以进行拼接
data_bal = np.expand_dims(data_bal, axis=0) #expand one dimension to concatenate
# 创建一个用于存储标签的初始数组，形状为 (1,)
data_lab = np.ones(1) #create array to fill labels

# 遍历每个标签区间
for i in num:
  # 注意这个 i 是区间右端点
  # 从标签中选择属于当前区间的索引
  # np.where() 返回的是一个元组，元组的元素个数取决于判断条件中的数据的维度, 元组的每个元素都是一个 数组，这些数组表示满足条件的元素在原始数组中的索引。因此，需要通过 indices[0] 访问索引数组
  # 如果输入数组是 多维的，返回的元组会包含 每一维的索引数组。例如，若数组是三维的，返回的元组就会包含三个数组，分别表示满足条件的元素在三维空间中每一维的索引。
  indices = np.where((labels > i-3) & (labels <= i)) #select indcies from every 3 meter interval until 36
  # 在当前区间中随机抽样 800 个样本
  # sampled_indices = np.random.choice(indices[0].flatten(), size=sample_size, replace=False) #random sample of each interval
  sampled_indices = np.random.choice(indices[0], size=sample_size, replace=False)
  # 提取对应的特征和标签
  tempx = features[sampled_indices]
  tempy = labels[sampled_indices]
  # 将当前区间的特征和标签拼接到平衡数组中
  data_bal = np.concatenate((data_bal, tempx), axis=0)
  data_lab = np.concatenate((data_lab, tempy), axis=0)

# 处理 labels > 36 的标签，这部分直接拼接
indices = np.where((labels > 36)) #add the values > 36 m, they are so few no sample needed
# sampled_indices = indices[0].flatten()
sampled_indices = indices[0]
tempx = features[sampled_indices]
tempy = labels[sampled_indices]
data_bal = np.concatenate((data_bal[1:], tempx), axis=0)
data_lab = np.concatenate((data_lab[1:], tempy), axis=0)

# Neural Network

如果你需要经过数据平衡之后的数据集应该在模型中使用特征数据 data_bal 和 标签数据 data_lab 来进数据集的和划分,而不是使用 features 和 labels 来进行数据集的划分(因为features 和 labels 未经过上采样)

### Training and evaluation

划分训练集和测试集

In [ ]:
# features 数组形状为 (num_samples, 90, 5, 5)，意味着每个样本有 90 个特征（或 90 个通道），每个特征是一个 5x5 的空间窗口。
# axis=(2, 3) 表示对每个特征的 5x5 窗口进行平均。结果是将每个 5x5 窗口压缩为一个标量（均值）
# features_mean 是一个二维数组，每个样本有 90 个特征（每个特征是原来 5x5 窗口的平均值）。
# 将每个特征的 5x5 窗口进行均值处理
features_mean = np.mean(features, axis=(2, 3)) # patch mean of size * size features
# 现在 features_mean 的形状是 (num_samples, 90)，适用于NN或者其他传统机器学习算法
# 注意: train_test_split 只能处理 NumPy 数组或 Pandas DataFrame，并不能直接处理 TensorFlow Dataset 对象。因此，这部分代码在处理 TensorFlow Dataset 时会出错。
X_train, X_test, y_train, y_test = train_test_split(features_mean, labels , test_size = 0.3, random_state=3)


In [ ]:
# %load_ext 是 Jupyter Notebook 中的一个魔法命令，用于加载扩展。
# tensorboard 是 TensorFlow 提供的一个工具，用于可视化训练过程中的数据，如损失函数、准确率、权重更新等。
# 在加载扩展后，你可以在 Jupyter Notebook 中直接使用 %tensorboard 命令来启动 TensorBoard。 在代码的 %tensorboard --logdir logs 部分启动了 TensorBoard
%load_ext tensorboard


神经网络模型定义

In [ ]:
# 使用 Sequential 模型，适用于线性堆叠的神经网络。
modelNn = Sequential()  # build neural network

# 第一层：Dense(128, input_shape=(90,), ...)代表输入层，输入形状为 90,)，表示每个输入样本有 90 个特征。这个 90 是硬编码的，但实际上它应该等于特征的维度数，这里可能需要根据实际情况进行修改。
# 这个形状定义的前提是你的特征数据 features 确实有 90 个特征（即形状为 (num_samples, 90)）。你需要确保实际的输入数据与定义的输入层形状一致。
# 如果特征数是动态变化的，建议不要硬编码这个数字，而是通过 features.shape[1] 获取动态的特征维度。
# ernel_initializer='normal' 表示权重的初始化方式为正态分布
# activation='relu' 使用 ReLU 激活函数。
modelNn.add(Dense(128, input_shape=(90,), kernel_initializer='normal', activation='relu'))
# modelNn.add(Dense(128, input_shape=(features.shape[1],), kernel_initializer='normal', activation='relu'))

# 第二层和第三层：分别包含 256 个神经元，也是全连接层，使用 ReLU 激活函数。
modelNn.add(Dense(256, kernel_initializer='normal', activation='relu'))
modelNn.add(Dense(256, kernel_initializer='normal', activation='relu'))
modelNn.add(Dense(128, kernel_initializer='normal', activation='relu'))

# 使用 Dropout(0.4) 防止过拟合，随机丢弃 40% 的神经元
modelNn.add(Dropout(0.4))

# 输出层包含 1 个神经元，activation='linear' 表示线性激活函数，适用于回归任务。
modelNn.add(Dense(1, kernel_initializer='normal', activation='linear'))


# 打印模型的结构，包括每一层的类型、输出形状、参数数量等信息，帮助你了解模型的结构。
modelNn.summary()


 模型编译

In [ ]:
# 损失函数：loss='mean_absolute_error' 使用绝对误差作为回归问题的损失函数。
# 优化器：optimizer='adam' 使用 Adam 优化器，它是目前常用的一种高效的优化算法。
# 评估指标：metrics=['mean_absolute_percentage_error'] 使用平均绝对百分比误差（MAPE）作为评估指标。

# 注意: 在训练过程中，优化算法会根据 val_loss 来更新模型的参数，因为 val_loss 是损失函数的值，而损失函数通常是模型优化的目标。
#    MAPE 作为评估指标，不会直接影响模型的参数更新，它仅用于评价模型在验证集上的相对误差，帮助你了解模型的实际表现。
modelNn.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_percentage_error']) #compile model


回调函数设置

In [ ]:
# 生成一个包含当前时间戳的日志目录路径。为 TensorBoard 准备日志文件存储位置。

# os.path.join("logs", ...) 用于将 logs 目录和时间戳连接成完整的路径。
# datetime.datetime.now() 获取当前的日期和时间。
# .strftime("%Y%m%d-%H%M%S") 是日期时间格式化方法，将当前时间格式化为 YYYYMMDD-HHMMSS 形式的字符串（例如，20231230-140102），这样可以确保每次训练的日志目录都不同，避免覆盖。
# 结果是一个路径，如 logs/20231230-140102，这个路径会作为 TensorBoard 的日志目录，用于存储当前训练的所有日志文件。
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))  # log directory for tensorboard

# 在训练过程中将日志信息保存到指定的 logdir 目录，并设置每个 epoch 保存权重的直方图。在训练过程中启用 TensorBoard 回调，确保训练日志被记录。
# 可视化训练过程中的日志信息，logdir 为日志文件夹的路径。histogram_freq=1 表示每个 epoch 保存直方图。
tensorboard_callback = TensorBoard(logdir, histogram_freq=1)

# 模型保存, 仅保存表现最好的模型。
# 验证集的损失（val_loss） 是指模型在验证集上的损失函数的值。损失函数度量了模型在验证集上的预测与实际标签之间的差距。损失函数在训练过程中用来指导模型的优化过程。
# ModelCheckpoint 是 Keras 提供的一个回调函数，用于在训练过程中根据某个指定的条件保存模型。默认情况下，ModelCheckpoint 会监控模型的 val_loss（验证集的损失）。
# save_best_only=True 表示只有在验证集的性能（如验证损失或验证准确率,默认情况下是验证集的损失）比之前的最小值还小，才会保存模型的权重。否则，当前的模型会被忽略。
# 这个回调函数会在每个 epoch 结束后检查模型的性能，并根据设置的条件决定是否保存当前模型。
model_save = ModelCheckpoint("/content/gdrive/MyDrive/NNmodels/best_model3.hdf52", save_best_only=True)  # directory for best model


模型训练

In [ ]:
# 使用 model.fit() 训练模型：
# Xtrain 和 ytrain 是训练集的特征和标签。
# epochs=100 训练 100 轮。
# validation_data=(Xtest, ytest) 在每一轮训练后，使用测试集 Xtest 和 ytest 进行验证。
# allbacks=[tensorboard_callback, model_save] 传入回调函数，分别用于 TensorBoard 可视化和保存最好的模型。
modelNn.fit(Xtrain, ytrain, epochs = 100, validation_data=(Xtest, ytest), callbacks=[tensorboard_callback, model_save]) #fit model

加载最好的模型

In [ ]:
bmodel = load_model('/content/gdrive/MyDrive/NNmodels/best_model3.hdf52') #load best model

 模型预测

In [ ]:
# 使用训练好的模型对测试集 Xtest 进行预测，返回预测值 ypred_nn。
ypred_nn = bmodel.predict(Xtest)


模型评估

In [ ]:
# 计算测试集的均方误差。 (MSE)
mse_nn = mse(ytest, ypred_nn)
# 计算均方根误差 (RMSE)
rmse_nn = mse_nn ** (1/2)
# 计算平均绝对误差 (MAE)
mae_nn = mae(ytest, ypred_nn)
# 平均绝对百分比误差 (MAPE)
mape_nn = mape(ytest, ypred_nn)

# 打印出 MAPE、MAE 和 RMSE 评估指标，帮助你评估模型的表现
print(mape_nn)
print(mae_nn)
print(rmse_nn)


In [ ]:
# 在 Jupyter Notebook 中查看训练过程中的各种指标。
# 启动 TensorBoard 并将其指向训练日志目录（logs）。
# %tensorboard 是 Jupyter 魔法命令，用于启动 TensorBoard 可视化工具。
# --logdir 是一个命令行参数，用于指定 TensorBoard 要读取的日志文件夹。这个目录包含了你在训练过程中生成的所有日志文件，TensorBoard 会通过读取这些日志文件来展示模型训练的过程。
# logs 目录是用来存放训练过程中记录的所有日志文件的地方。通过使用 --logdir logs 参数，TensorBoard 会读取这个目录下的文件，从而显示出训练过程中的各种可视化图表。
%tensorboard --logdir logs

# 在模型训练中：你首先使用 tensorboard_callback（带有 logdir 的回调）将训练过程中的日志信息保存到指定的目录。
# 在训练后可视化：在训练结束后，你可以使用 %tensorboard --logdir logs 来启动 TensorBoard 可视化工具，查看训练过程中的详细信息，如损失曲线、准确率曲线、权重分布等。
# 这两个部分配合使用，TensorBoard 回调用于训练时记录数据，而 %tensorboard 命令用于启动可视化界面以便查看。

Quantile(分位数) 和统计信息

In [ ]:
# 计算预测结果的均值
mean_nn = np.mean(ypred_nn[:])  # calculate mean
# 计算预测结果的不同分位数（1%, 25%, 50%, 75%, 99%）
quantiles_nn = np.percentile(ypred_nn[:], [1, 25, 50, 75, 99])  # calculate quantiles 0.01, 0.25, 0.5, 0.75, 0.99

# 计算标签的均值
mean_labels = np.mean(labels[:])
# 计算标签的不同分位数（1%, 25%, 50%, 75%, 99%）
quantiles_labels = np.percentile(labels[:], [1, 25, 50, 75, 99])

print(mean_nn)
print(quantiles_nn)
# 打印预测值中最小的 10 个和最大的 10 个。
print(np.sort(ypred_nn.flatten())[:10])  # print the 10 lowest predictions
print(np.sort(ypred_nn.flatten())[-10:][::-1])  # print the 10 highest predictions

print(mean_labels)
print(quantiles_labels)
# 打印标签中最小的 10 个和最大的 10 个。
print(np.sort(labels.flatten())[:10])
print(np.sort(labels.flatten())[-10:][::-1])


## Prediction of test image neural network

有专门准备测试图像才用得上这部分代码,没有测试图像就用不上了

测试图像被分割为1024*1024的大小，通道数为90，即分割得的图像块形状为（90，1024，1024）

以后有新的图像数据需要进行预测，那么可以参考这部分代码来处理这些新数据，提取图像块并进行预测。

加载测试数据并切分为图像块（patches）

In [ ]:
#加载测试图像
# Xs 是包含测试图像的数组，形状假设为 (channels, height, width)
Xs = np.load('/content/gdrive/MyDrive/TestData/private_test_image_reduced.npy')  # load test image
# 定义图像块大小
patch_size = 1024  # define patch size
# 初始化图像块索引
ind = 0  # index for saved patch images

# 遍历图像并切分成大小为 patch_size 的图像块
# range(4) 相当于 range(0, 4), 也就是起始值为 0，终止值为 4（但 4 不包含在内）。
# 通过两层 for 循环，图像被分割为 4x4 块，每块大小为 1024x1024。每个图像块被单独保存为 .npy 文件。
for i in range(4):
  for j in range(4):
    temp = Xs[:,i*patch_size:(i+1)*patch_size, j*patch_size:(j+1)*patch_size]
    # 使用 np.save() 将每个图像块保存到 TestPatches 文件夹中。ind 是保存文件时的索引，确保文件名唯一。
    np.save('/content/gdrive/MyDrive/TestPatches/patch_' + str(ind) + '.npy', temp)
    # 更新图像块索引
    ind = ind + 1


加载训练好的神经网络模型并预测

In [ ]:
# 加载保存由测试图像切割而来的图像块的文件夹路径
folder_path = '/content/gdrive/MyDrive/TestPatches/'  #folder path
# 加载已训练的神经网络模型
nnmodel = load_model('/content/gdrive/MyDrive/NNmodels/best_model3.hdf52')
# ind 是保存由测试图像切割得的图像块文件时的索引，以确保文件名唯一。
ind = 0

# Iterate over the files in the folder
# 遍历文件夹中的图像块
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
      # 加载图像块
      # X 的形状应该是 (90, 1024, 1024)。
      X = np.load(file_path) #load patch

      # 网络的输入应该是 (batch_size, feature_size)，因此应该将 X 直接展平为 (batch_size, 90)。这样才符合神经网络的输入要求（即每个样本有 90 个特征）。
      # 将图像块重塑为 (90, -1) 形状，并转置. 即将 X 重塑为了(90, 1024 * 1024), 转置以后形状为(1024 * 1024, 90)
      # 此时每一行代表一个像素点的所有特征（即每个像素的 90 个波段的值），而列数代表特征的维度。这样做的原因是因为神经网络的输入需要每个样本是一个向量，每个样本代表一个像素点的 90 个特征。这样处理后的数据可以直接传入神经网络。
      Xr = X.reshape(90,-1).transpose() #transpose patch

      # 使用神经网络进行预测
      # 神经网络预测时需要一个二维矩阵，其中每行是一个样本（像素点）的特征，形状是 (batch_size, 90)，其中 batch_size 是像素点的数量（即 size * size），而 90 是特征的维度。
      # nnpred 的形状应该是 (size * size, 1)的二维数组，表示每个像素的预测值, 而不是 (size * size,)的一维数组。这一点也可以从后一句"将预测结果重塑为图像块大小"的代码看出.
      nnpred = nnmodel.predict(Xr) #predict labels
      # 将预测结果重塑为图像块大小
      # transpose() 会将 nnpred 从 (size * size, 1) 转换为 (1, size * size)。
      # .reshape(1, 1024, 1024) 会将其重塑为一个大小为 (1, 1024, 1024) 的三维数组，这表示你正在把预测结果重新调整为图像块的形状。
      nnpredr = nnpred.transpose().reshape(1,1024,1024) #reshape patch to image size
      # 保存预测结果
      np.save('/content/gdrive/MyDrive/MaskNN/mask_'+ str(ind) + '.npy', nnpredr) #save patch
      # 更新索引
      ind = ind + 1
      # 输出预测结果的分位数
      print(np.percentile(nnpred[:], [1, 25, 50, 75, 99]))
      # 输出最小的 10 个预测值
      print(np.sort(nnpred.flatten())[:10])
      # 输出最大的 10 个预测值
      print(np.sort(nnpred.flatten())[-10:][::-1])

重建预测图像并保存：

In [ ]:
# 预测结果文件夹路径
folder_path = '/content/gdrive/MyDrive/MaskNN/'  #folder path
img_list = []

# Iterate over the files in the folder
# 遍历文件夹中的预测图像块
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        # Load the data from the file
        data = np.load(file_path) #加载预测结果
        # Append the data to the list
        img_list.append(data)  #将预测结果添加到列表中

# 将每四个图像块按列(宽度方向)拼接
# Concatenate the patches along the columns (horizontal axis)
im1 = np.concatenate((img_list[0], img_list[1], img_list[2], img_list[3]), axis=2)
im2 = np.concatenate((img_list[4], img_list[5], img_list[6], img_list[7]), axis=2)
im3= np.concatenate((img_list[8], img_list[9], img_list[10], img_list[11]), axis=2)
im4 = np.concatenate((img_list[12], img_list[13], img_list[14], img_list[15]), axis=2)

# 再将四个拼接好的部分按行(高度方向)拼接
# Concatenate the rows along the vertical axis to rebuild the original image
original_image = np.concatenate((im1, im2, im3, im4), axis=1)

#保存重建后的完整预测图像
np.save('/content/gdrive/MyDrive/FinalPredictions/mask_private_nn.npy', original_image)


可视化预测结果

In [ ]:
plot_rebuild_image = np.load('/content/gdrive/MyDrive/FinalPredictions/mask_private_nn.npy')
# 将重建的预测图像由 (1, height, width) 转换为 (height, width)
tree_height_2d = plot_rebuild_image[0]

# 使用 `matplotlib` 绘制图像
# Plot the tree height data
plt.imshow(tree_height_2d, cmap='viridis')  # 使用 `viridis` 色图

# Add colorbar for reference
plt.colorbar()  # 添加颜色条

# Display the plot
plt.show()

输出预测结果的分位数和极值

In [ ]:
print(np.percentile(plot[:], [1, 25, 50, 75, 99])) #calculate quantiles 0.01, 0.25, 0.5, 0.75, 0.99

print(np.sort(plot.flatten())[:10]) #print the 10 lowest predictions
print(np.sort(plot.flatten())[-10:][::-1]) #print the 10 highest predictions

# Convolutional neural network

### Training and evaluation

划分训练集和测试集

In [ ]:
# 数据集拆分：训练集和测试集
Xtrain, Xtest, ytrain, ytest = train_test_split(features, labels , test_size = 0.3, random_state=3) #create train, test set


In [ ]:
# 加载 TensorBoard 插件，用于可视化训练过程的日志信息，如损失曲线、指标等。
%load_ext tensorboard

卷积神经网络模型定义

In [ ]:
# Keras 默认使用 channels_last 格式，这表示图像的输入形状是 (height, width, channels)。如果想使用 channels_first 格式（即 (channels, height, width)），需要进行相应的设置，比如通过 Keras 配置全局设置或者在模型层中明确指定
# 这里采用为每个层指定数据格式。只有卷积层和池化层需要明确指定数据格式（channels_first 或 channels_last），而全连接层不需要这样做，展平层会处理掉格式问题。
# 卷积操作会根据指定的数据格式进行计算，输出的形状也会遵循这种数据格式。

# 使用 Sequential() 创建一个顺序模型（即按顺序堆叠各层）。
modelCnn = Sequential() #bulid cnn

# 因为 features 数组形状为 (num_samples, 90, 5, 5)
# 指定输入数据的形状。这里假设每个输入图像是一个 90 个通道的 5x5 大小的图像块。
modelCnn.add(InputLayer(input_shape = (90, 5, 5))) # 输入形状为 (channels, height, width)

# 注意：在卷积神经网络中，卷积核的形状是 (高度, 宽度, 输入通道数, 输出通道数)，这代表每个卷积核的尺寸、它需要处理的输入通道数，以及它生成的输出通道数。
# 高度和宽度决定了卷积核的空间大小，常见的卷积核大小有 3x3、5x5、7x7 等。
# 输入通道数是卷积核需要处理的输入特征图的深度（即输入图像的通道数）。
# 输出通道数是卷积层的过滤器数目，也就是卷积层最终生成的特征图的数量。

# 设置 Conv2D 使用 channels_first 格式

# 当你定义一个卷积层时，卷积核的数量会决定卷积层输出的通道数，但卷积核的深度（即每个卷积核的输入通道数）是由输入数据的通道数（即输入特征图的深度）来决定的，而不用手动设置。
# Conv2D 添加一个卷积层，filters=128 表示该层将有 128 个卷积核（即输出通道数），每个卷积核对应一个输出通道。
# kernel_size=(3,3) 表示卷积核的大小是 3x3。trides=1 表示步幅为 1，即卷积核每次移动 1 个像素。padding="same" 表示零填充（zero padding）策略，目的是使得 卷积操作后 输出特征图的尺寸 保持与输入相同（即宽度和高度保持不变）。activation='relu' 使用 ReLU 激活函数。
# 第一层卷积核的形状是 (3, 3, 90，128)，该卷积层的输出图像的形状为(128, 5, 5)，其中 128 是输出的通道数，由于设置了padding = "same"，故输出图像大小不变。
modelCnn.add(Conv2D(filters=128, kernel_size= (3,3), strides=  1 , padding = "same", activation='relu', data_format='channels_first'))

# 另一个卷积层，filters=256 代表使用 256 个卷积核（即输出通道数），每个卷积核对应一个输出通道。padding="valid" 表示不使用填充，输出大小会减少。
# 第二层卷积核的形状是 (3, 3, 128，256)，该卷积层的输出图像的形状为(256, 3, 3)，其中 256 是输出的通道数，由于使用了 valid padding，输出尺寸减少。
modelCnn.add(Conv2D(filters=256, kernel_size= (3,3), strides=  1 , padding = "valid", activation='relu', data_format='channels_first'))

# MaxPool2D 添加一个最大池化层，pool_size=(2,2) 表示 2x2 的池化窗口，池化操作的 步长 默认是 2，减少空间维度（降低特征图的尺寸）。
# 用 2x2 的过滤器，以2为步长进行特征值提取，因此该池化操作会将输入特征图的 空间尺寸（宽度和高度） 缩小一半。采用的池化操作是 最大池化。
# 输入尺寸为 (256, 3, 3)，经过池化后，输出的尺寸会变为 (256, 1, 1)，因为池化操作会将 3x3 的特征图缩小为 1x1。
modelCnn.add(MaxPool2D(pool_size = (2,2), data_format='channels_first'))

# Flatten() 层将二维的特征图展平为一维向量，为全连接层做准备。
# Flatten() 会将池化层输出的 (256, 1, 1) 转换为 256 的一维向量。这是因为在全连接层之前，必须将输入转换为一维数据。
modelCnn.add(Flatten())

# 全连接层处理的是一维向量，而不是多维的图像数据。所以全连接层的输入格式不依赖于 channels_first 或 channels_last。
# Dense(512) 添加一个全连接层，包含 512 个神经元，activation='relu' 使用 ReLU 激活函数。
modelCnn.add(Dense(512, activation='relu'))

# 另一个全连接层，包含 128 个神经元。
modelCnn.add(Dense(128, activation='relu'))

# Dropout(0.4) 是一个正则化方法，随机丢弃 40% 的神经元，防止过拟合。
modelCnn.add(Dropout(0.4))

# 最后一层是一个包含 1 个神经元的全连接层，使用线性激活函数（适合回归任务）。
modelCnn.add(Dense(1, activation='linear'))


# summary() 输出模型的概况，显示每一层的参数数量和输出形状。
modelCnn.summary()

模型编译

In [ ]:
# 使用 compile() 方法指定模型的损失函数、优化器和评估指标。
# 使用平均绝对误差作为损失函数. 使用 Adam 优化器. 使用平均绝对百分比误差作为评估指标。
modelCnn.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_percentage_error']) #compile cnn


回调函数设置

In [ ]:
# 设置日志文件夹路径。
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
# TensorBoard 回调用于在训练过程中记录日志，供 TensorBoard 使用。
# histogram_freq=1 表示每个 epoch 都记录权重的直方图
tensorboard_callback = TensorBoard(logdir, histogram_freq=1)

# ModelCheckpoint 回调用于在验证集损失最小化时保存最佳模型。
model_save = ModelCheckpoint("/content/gdrive/MyDrive/NNmodels/best_modelCnnPred.hdf5", save_best_only = True) #save best model


模型训练

In [ ]:
# 指定训练数据、测试数据、训练轮次（epochs=100）以及回调函数（tensorboard_callback 和 model_save）。
modelCnn.fit(Xtrain, ytrain, epochs = 100, validation_data=(Xtest, ytest), callbacks=[tensorboard_callback, model_save]) #train cnn


加载最好的模型

In [ ]:

bmodel = load_model('/content/gdrive/MyDrive/NNmodels/best_modelCnnPred.hdf5')

模型预测

In [ ]:

ypred_cnn = bmodel.predict(Xtest) #predict best model

模型评估

In [ ]:
# 计算均方误差（MSE）、均方根误差（RMSE）、平均绝对误差（MAE）和平均绝对百分比误差（MAPE）。
# 这些指标用于评估模型在测试集上的性能。
mse_cnn = mse(ytest, ypred_cnn) #calculate metrics
rmse_cnn = mse_cnn ** (1/2)
mae_cnn = mae(ytest, ypred_cnn)
mape_cnn = mape(ytest, ypred_cnn)

print(mape_cnn)
print(mae_cnn)
print(rmse_cnn)

In [ ]:
# 启动 TensorBoard 可视化工具，查看训练过程中的详细信息，如损失曲线、准确率曲线、权重分布等。
%tensorboard --logdir logs

Quantile(分位数) 和统计信息

In [ ]:
# ypred_cnn[:] 和 labels[:] 都是对数组的切片操作，表示返回数组中的所有元素，分别是预测值和真实标签值。
# 无论数组的维度是多少，使用 [:] 都是返回数组中的所有元素。这是 NumPy 中的一个常见用法，它用于获取数组的完整内容，不论数组是多维的。(保持其原有的形状。)

# 计算预测结果的均值
# ypred_cnn 的形状应为 (n, 1)，即一个列向量，每一行代表一个样本的预测结果。如果你打印 ypred_cnn[:]，你将得到模型对所有测试样本的预测结果。
mean_cnn = np.mean(ypred_cnn[:]) #calculate mean
# 计算预测结果的分位数
quantiles_cnn = np.percentile(ypred_cnn[:], [1, 25, 50, 75, 99]) #calculate quantiles 0.01, 0.25, 0.5, 0.75, 0.99

# 计算标签的均值
# labels 是一个形状为 (n,) 的一维数组，存储了对应测试样本的实际标签（真实的森林高度）。
# labels[:] 会返回所有的标签值。
mean_labels = np.mean(labels[:])
# 计算标签的分位数
quantiles_labels = np.percentile(labels[:], [1, 25, 50, 75, 99])

print(mean_cnn)
print(quantiles_cnn)
# 打印预测值中最小的 10 个和最大的 10 个。
print(np.sort(ypred_cnn.flatten())[:10]) #print the 10 lowest predictions
print(np.sort(ypred_cnn.flatten())[-10:][::-1]) #print the 10 highest predictions

print(mean_labels)
print(quantiles_labels)
# 打印标签中最小的 10 个和最大的 10 个。
print(np.sort(labels.flatten())[:10])
print(np.sort(labels.flatten())[-10:][::-1])

### Prediction of test image convolutional neural network

这里使用的直接是前面NN模型中为了对测试图像进行预测从而对测试图像进行分割所得的测试图像块，测试图像的大小为1024*1024，通道数为90，即形状为（90，1024，1024）。

所以下面的代码中 X 的形状才为（90，1024，1024）。

加载测试数据并切分为图像块（patches），并使用训练好的神经网络模型并预测

In [ ]:
# 设置存储图像块的文件夹路径
folder_path = '/content/gdrive/MyDrive/TestPatches/'  # 文件夹路径，存放测试图像块
# 加载训练好的模型
cnnmodel = load_model('/content/gdrive/MyDrive/NNmodels/best_modelCnnPred.hdf5')  # 加载已经训练好的CNN模型
# 初始化索引变量，用于保存预测结果
ind = 0

# Iterate over the files in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
      X = np.load(file_path)

      # 因为卷积神经网络通常需要固定大小的输入，为了符合网络输入要求，这里对图像块 X 零填充
      # 这里的 pad_width 是一个 元组，它描述了三个维度的填充方式。即在通道维度上不进行填充，图像的通道数不改变，维度保持不变。在第二和第三个轴上(即 高度维度 和 宽度维度)进行宽度为 2 的填充, 填充的大小为 (2, 2)，意味着在每个图像高度的上方和下方和宽度的左侧和右侧各添加 2 行/列的零。因此，图像的高度会增加 4 个像素, 图像的宽度会增加 4 个像素。这样做的目的是为了保证卷积操作的窗口大小不变。
      # Pad width of 2 on the second and third axes to (90, 1028, 1028) to get output of shape (90, 1024, 1024) with a window of size 5
      pad_width = ((0, 0), (2, 2), (2, 2))  # Pad width of 2 on the second and third axes to (10, 1028, 1028) to get output of shape (10, 1024, 1024) with a window of size 5
      # mode='constant' 表示使用常数值填充。constant_values=0 指定填充的常数值为 0。
      # 经过填充后的图像形状将变为 (90, 1028, 1028)。
      X = np.pad(X, pad_width, mode='constant', constant_values=0)

      # 获取图像的高度和宽度, 也就可以看出这个 X 的形状为(channels, height, width)
      height, width = X.shape[1], X.shape[2] #get height an width of the image

      # 提取图像块的窗口
      # 定义窗口大小为 5x5
      window_size = 5
      # 定义窗口形状，90个通道（假设输入是90个通道），5x5窗口
      shape = (90, window_size, window_size)
      # 创建一个形状为 (90, 5, 5) 的数组，填充为1
      datat = np.ones(shape) #array for features
      # 扩展维度，使其成为 (1, 10, 5, 5)
      datat = np.expand_dims(datat, axis=0)

      # 对图像进行切割，提取滑动窗口

      # 选择第一行（90, 5, 5）的窗口（这里的第一行指的是将所有的图像块提取出来拼接后，这部分代码提取出来的就是该拼接图像的第一行图像）
      for w in range(width - window_size + 1): # select the first row (90, 5, 5) patches
        # 从图像中提取一个5x5的窗口
        # w 就是滑动窗口的左端点（注意是左端点，所以没问题），每次 w 增加 1，表示窗口向右滑动 1 个像素。
        sample = X[:, 0:window_size, w:w+window_size]
        sample2 = np.expand_dims(sample, axis=0)
        datat = np.concatenate((datat, sample2), axis=0)

      datat = datat[1:,:,:,:] #remove the first artificial patch

      # 遍历每一行
      # 对图像的每一行进行处理，提取窗口（注意：这里的每一行指的就不是刚才那个拼接窗口的图像了，而是指输入进行窗口切割的图像）
      # 这里height - window_size而不是height - window_size + 1 是因为上一段代码已经提取出来了第一行（90, 5, 5）的窗口，所以循环次数少1
      for h in range(height - window_size): # iterate over the rows
        # 创建一个形状为 (90, 1, 5) 的空数组
        shape = (90, 1, window_size)
        datat2 = np.ones(shape) #array for the values of one row
        # 扩展维度，使其成为 (1, 90, 1, 5)
        datat2 = np.expand_dims(datat2, axis=0)
        # 遍历每一列
        for w in range(width - window_size  + 1): # iterate over the columns
          # 提取窗口
          # sample 的形状是 (90, window_size)
          # 由于你只选择了一个高度索引（h+window_size），所以返回的是一个 (90, window_size) 的数组。90 是通道数，window_size 是窗口大小（即列数）
          # Python 和 NumPy 通常会“自动去掉”维度为 1 的通道（即，当某个维度的大小为 1 时，这个维度可能被省略）。
          sample = X[:, h+window_size, w:w+window_size]
          # 扩展维度，变为 (90, 1, window_size)
          sample = np.expand_dims(sample, axis=1)
          # 扩展维度，变为 (1, 90, 1, window_size)
          # 第一维度 1 代表了“批量大小”（batch size），意味着这个样本是一个单独的样本，且是批处理的一部分。
          # 第三维度 1 代表了窗口的高度（即对单行窗口的处理）。
          sample = np.expand_dims(sample, axis=0)
          # 将窗口添加到 datat2 数组中
          datat2 = np.concatenate((datat2, sample), axis=0) #collect values from one row

        # 拼接前一行的处理结果（即 datat）和当前行的窗口数据（即 datat2）。
        # 这个操作与处理行 h 相关，确保从前一行的结果中正确地选择出窗口数据。这里的 h 是当前行的索引。
        # 这部分代码确实没错，因为 datat 是最终存储拼接图像块的数组（注意这个拼接图像块是一行一行拼接的，不是一个一个拼接的），而这里构建每一行的图像块是采取的保留前一行图像块的后 size -1 行高度，然后再拼接新的当前高度的图像块。
        # 而保留前一行图像块，则需要将前面剩余行的图像块删去即使用 datat的(width-window_size+1)*h: 来实现。而保留前一块图像块中的后 size - 1 行高度，则使用 datat 的1: 来实现。
        datat4 = np.concatenate((datat[(width-window_size+1)*h:,:,1:,:], datat2[1:,:,:,:]), axis=2) #remove the first column from the previous row patches and add a new row below
        datat = np.concatenate((datat, datat4), axis=0) #stack the patches

      # 使用CNN模型进行预测
      # 如果您传入的 datat 形状是 (n, 90, 5, 5)，则经过卷积层和池化层后，最终输出的形状为 (n, 1)，其中 n 是样本的数量（即输入数据的批大小）。
      # 返回的预测结果是一个形状为 (num_samples, 1) 的数组，其中每一行是一个预测的标量值。
      pred = cnnmodel.predict(datat) #predict labels
      # 重塑预测结果为原始图像大小
      img = pred.transpose().reshape(1,1024,1024) #reshape image to original size
      # 保存预测结果
      np.save('/content/gdrive/MyDrive/MaskCNN/mask_'+ str(ind) + '.npy', img)
      # 更新索引
      ind = ind + 1
      # 打印预测图像的形状
      print(img.shape) #metrics to control the prediction process for every patch image
      # 输出预测值的分位数
      print(np.percentile(pred[:], [1, 25, 50, 75, 99]))
       # 输出最小的10个预测值
      print(np.sort(pred.flatten())[:10])
      # 输出最大的10个预测值
      print(np.sort(pred.flatten())[-10:][::-1])


重建预测图像并保存

In [ ]:
folder_path = '/content/gdrive/MyDrive/MaskCNN/'  #folder path
img_list = []

# Iterate over the files in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        # Load the data from the file
        data = np.load(file_path)
        # Append the data to the list
        img_list.append(data)
# Concatenate the patches along the columns (horizontal axis)
im1 = np.concatenate((img_list[0], img_list[1], img_list[2], img_list[3]), axis=2)
im2 = np.concatenate((img_list[4], img_list[5], img_list[6], img_list[7]), axis=2)
im3= np.concatenate((img_list[8], img_list[9], img_list[10], img_list[11]), axis=2)
im4 = np.concatenate((img_list[12], img_list[13], img_list[14], img_list[15]), axis=2)

# Concatenate the rows along the vertical axis to rebuild the original image
original_image = np.concatenate((im1, im2, im3, im4), axis=1)

np.save('/content/gdrive/MyDrive/FinalPredictions/mask_private_cnn.npy', original_image)


可视化预测结果

In [ ]:
plot = np.load('/content/gdrive/MyDrive/FinalPredictions/mask_private_cnn.npy')
# 将重建的预测图像由 (1, height, width) 转换为 (height, width)
tree_height_2d = plot[0]

# Plot the tree height data
plt.imshow(tree_height_2d, cmap='viridis')

# Add colorbar for reference
plt.colorbar() # 添加颜色条

# Display the plot
plt.show()

输出预测结果的分位数和极值

In [ ]:
print(np.percentile(plot[:], [1, 25, 50, 75, 99])) #calculate quantiles 0.01, 0.25, 0.5, 0.75, 0.99

print(np.sort(plot.flatten())[:10]) #print the 10 lowest predictions
print(np.sort(plot.flatten())[-10:][::-1]) #print the 10 highest predictions